In [1]:
import pandas as pd

from src import bounded_mean_1d, nb_setup

nb_setup()

### Create raw dataframe

In [2]:
# Import data
df = pd.read_csv("../../../data/par2_nebd_embryos.csv")
df_quantification = pd.read_csv("../../../data/par2_nebd_quantification.csv")

# Filter relevant data
lines = ["lp637", "nwg201", "nwg338", "nwg369"]
df = df[df.Line.isin(lines)]
df_quantification = df_quantification[df_quantification.EmbryoID.isin(df.EmbryoID)]


def calculate_summary(group):
    cyt = group["Cytoplasmic signal"].mean()
    mem_tot = group["Membrane signal"].mean()
    mem_ant = bounded_mean_1d(group["Membrane signal"], [0.4, 0.6])
    mem_post = bounded_mean_1d(group["Membrane signal"], [0.9, 0.1])
    return pd.Series(
        {
            "Cyt": cyt,
            "Mem_tot": mem_tot,
            "Mem_ant": mem_ant,
            "Mem_post": mem_post,
        }
    )


df_summary = (
    df_quantification.groupby("EmbryoID").apply(calculate_summary).reset_index()
)
df = pd.merge(df, df_summary, on="EmbryoID")

# Add uniform vs polarised column
unipol_dict = {line: "Pol" if line in ["lp637", "nwg338"] else "Uni" for line in lines}
df["UniPol"] = df["Line"].map(unipol_dict)

# Add genotype column
genotype_dict = {
    line: "WT" if line in ["lp637", "nwg201"] else "L109R" for line in lines
}
df["Genotype"] = df["Line"].map(genotype_dict)

# # Remove points below threshold
# mem_thresh = 4e2
# cyt_thresh = 2e2
# df = df[df.Mem_post > mem_thresh]
# df = df[df.Cyt > cyt_thresh]

### Create dataframes with absolute concentrations

In [3]:
def create_dataframe(D, c, df):
    # D = diameter of molecules (um)
    # c = cytoplasmic concentration in wt cells (M)
    # df = input dataframe
    df_copy = df.copy()
    wt_condition = (df.Line == "lp637") & (df.RNAi == "wt")
    mem_norm = df_copy.loc[wt_condition, "Mem_post"].mean()
    cyt_norm = df_copy.loc[wt_condition, "Cyt"].mean()
    xc = c
    xm = (mem_norm / cyt_norm) * (1 / D) * c
    for col in ["Mem_tot", "Mem_ant", "Mem_post", "Cyt"]:
        df_copy[col] *= xm / mem_norm if "Mem" in col else xc / cyt_norm
    return df_copy

In [4]:
# Main figure: D=5nm, c=10.4nM
df_main = create_dataframe(0.005, 1.04e-8, df)
df_main.to_csv("df_main.csv")

# Supplement 1: D=0.5nm, c=10.4nM
df_s1 = create_dataframe(0.0005, 1.04e-8, df)
df_s1.to_csv("df_s1.csv")

# Supplement 2: D=50nm, c=10.4nM
df_s2 = create_dataframe(0.05, 1.04e-8, df)
df_s2.to_csv("df_s2.csv")

# # Supplement 3: D=5.451nm, c=4.1nM
# df_s3 = create_dataframe(0.005451, 4.1e-9, df)
# df_s3.to_csv('df_s3.csv')

# # Supplement 4: D=5.451nm, c=410nM
# df_s4 = create_dataframe(0.005451, 4.1e-7, df)
# df_s4.to_csv('df_s4.csv')